All Techniques Of Hyper Parameter Optimization
1.GridSearchCV

2.RandomizedSearchCV

3.Bayesian Optimization -Automate Hyperparameter Tuning (Hyperopt)

4.Sequential Model Based Optimization(Tuning a scikit-learn estimator with skopt)

5.Optuna- Automate Hyperparameter Tuning

6.Genetic Algorithms (TPOT Classifier)

In [27]:
import warnings
warnings.filterwarnings('ignore')

In [28]:
import pandas as pd
df=pd.read_csv(r'C:\Users\PALLAW-PC\Desktop\ML&Deep_Learning\Diabetes_dataset\diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [29]:
import numpy as np
df['Glucose']=np.where(df['Glucose']==0,df['Glucose'].median(),df['Glucose'])
df['Glucose']=np.where(df['Insulin']==0,df['Insulin'].median(),df['Insulin'])
df['Glucose']=np.where(df['SkinThickness']==0,df['SkinThickness'].median(),df['SkinThickness'])
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,35.0,72,35,0,33.6,0.627,50,1
1,1,29.0,66,29,0,26.6,0.351,31,0
2,8,23.0,64,0,0,23.3,0.672,32,1
3,1,23.0,66,23,94,28.1,0.167,21,0
4,0,35.0,40,35,168,43.1,2.288,33,1


In [30]:
#### Independent And Dependent features
X=df.drop('Outcome',axis=1)
y=df['Outcome']

In [31]:
#### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=0)

In [32]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier=RandomForestClassifier(n_estimators=10).fit(X_train,y_train)
prediction=rf_classifier.predict(X_test)

In [33]:
y.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [34]:

from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[89 18]
 [29 18]]
0.6948051948051948
              precision    recall  f1-score   support

           0       0.75      0.83      0.79       107
           1       0.50      0.38      0.43        47

    accuracy                           0.69       154
   macro avg       0.63      0.61      0.61       154
weighted avg       0.68      0.69      0.68       154



The main parameters used by a Random Forest Classifier are:

#criterion = the function used to evaluate the quality of a split.
#max_depth = maximum number of levels allowed in each tree.
#max_features = maximum number of features considered when splitting a node.
#min_samples_leaf = minimum number of samples which can be stored in a tree leaf.
#min_samples_split = minimum number of samples necessary in a node to cause node splitting.
#n_estimators = number of trees in the ensamble.

In [35]:
### Manual Hyperparameter Tuning
model=RandomForestClassifier(n_estimators=300,criterion='entropy',
                             max_features='sqrt',min_samples_leaf=10,random_state=100).fit(X_train,y_train)
predictions=model.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(accuracy_score(y_test,predictions))
print(classification_report(y_test,predictions))

[[94 13]
 [28 19]]
0.7337662337662337
              precision    recall  f1-score   support

           0       0.77      0.88      0.82       107
           1       0.59      0.40      0.48        47

    accuracy                           0.73       154
   macro avg       0.68      0.64      0.65       154
weighted avg       0.72      0.73      0.72       154



In [36]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [37]:
rf=RandomForestClassifier()
rf_randomcv=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=100,cv=3,verbose=2,
                               random_state=100,n_jobs=-1)
### fit the randomized model
rf_randomcv.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  7.5min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 5, 10, 14],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=100, verbose=2)

In [38]:
rf_randomcv.best_params_

{'n_estimators': 2000,
 'min_samples_split': 14,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 340,
 'criterion': 'gini'}

In [39]:

rf_randomcv

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 5, 10, 14],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=100, verbose=2)

In [40]:
best_random_grid=rf_randomcv.best_estimator_

In [41]:
from sklearn.metrics import accuracy_score
y_pred=best_random_grid.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("Classification report: {}".format(classification_report(y_test,y_pred)))


[[90 17]
 [23 24]]
Accuracy Score 0.7402597402597403
Classification report:               precision    recall  f1-score   support

           0       0.80      0.84      0.82       107
           1       0.59      0.51      0.55        47

    accuracy                           0.74       154
   macro avg       0.69      0.68      0.68       154
weighted avg       0.73      0.74      0.73       154



In [42]:
rf_randomcv.best_params_

{'n_estimators': 2000,
 'min_samples_split': 14,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 340,
 'criterion': 'gini'}

# GridSearch CV

In [43]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion': [rf_randomcv.best_params_['criterion']],
    'max_depth': [rf_randomcv.best_params_['max_depth']],
    'max_features': [rf_randomcv.best_params_['max_features']],
    'min_samples_leaf': [rf_randomcv.best_params_['min_samples_leaf'], 
                         rf_randomcv.best_params_['min_samples_leaf']+2, 
                         rf_randomcv.best_params_['min_samples_leaf'] + 4],
    'min_samples_split': [rf_randomcv.best_params_['min_samples_split'] - 2,
                          rf_randomcv.best_params_['min_samples_split'] - 1,
                          rf_randomcv.best_params_['min_samples_split'], 
                          rf_randomcv.best_params_['min_samples_split'] +1,
                          rf_randomcv.best_params_['min_samples_split'] + 2],
    'n_estimators': [rf_randomcv.best_params_['n_estimators'] - 200, rf_randomcv.best_params_['n_estimators'] - 100, 
                     rf_randomcv.best_params_['n_estimators'], 
                     rf_randomcv.best_params_['n_estimators'] + 100, rf_randomcv.best_params_['n_estimators'] + 200]
}

print(param_grid)

{'criterion': ['gini'], 'max_depth': [340], 'max_features': ['sqrt'], 'min_samples_leaf': [1, 3, 5], 'min_samples_split': [12, 13, 14, 15, 16], 'n_estimators': [1800, 1900, 2000, 2100, 2200]}


In [44]:
#### Fit the grid_search to the data
rf=RandomForestClassifier()
grid_search=GridSearchCV(estimator=rf,param_grid=param_grid,cv=10,n_jobs=-1,verbose=2)
grid_search.fit(X_train,y_train)

Fitting 10 folds for each of 75 candidates, totalling 750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 16.6min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 28.0min
[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed: 32.3min finished


GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini'], 'max_depth': [340],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 3, 5],
                         'min_samples_split': [12, 13, 14, 15, 16],
                         'n_estimators': [1800, 1900, 2000, 2100, 2200]},
             verbose=2)

In [45]:
grid_search.best_estimator_

RandomForestClassifier(max_depth=340, max_features='sqrt', min_samples_split=14,
                       n_estimators=2200)

In [46]:
best_grid=grid_search.best_estimator_

In [47]:
best_grid

RandomForestClassifier(max_depth=340, max_features='sqrt', min_samples_split=14,
                       n_estimators=2200)

In [48]:
y_pred=best_grid.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("Classification report: {}".format(classification_report(y_test,y_pred)))


[[92 15]
 [23 24]]
Accuracy Score 0.7532467532467533
Classification report:               precision    recall  f1-score   support

           0       0.80      0.86      0.83       107
           1       0.62      0.51      0.56        47

    accuracy                           0.75       154
   macro avg       0.71      0.69      0.69       154
weighted avg       0.74      0.75      0.75       154



Automated Hyperparameter Tuning

Automated Hyperparameter Tuning can be done by using techniques such as

#Bayesian Optimization
#Gradient Descent
#Evolutionary Algorithms
#Bayesian Optimization
#Bayesian optimization 
uses probability to find the minimum of a function. The final aim is to find the input value to a function which can gives us the lowest possible output value.It usually performs better than random,grid and manual search providing better performance in the testing phase and reduced optimization time. In Hyperopt, Bayesian Optimization can be implemented giving 3 three main parameters to the function fmin.

#Objective Function = defines the loss function to minimize.
#Domain Space = defines the range of input values to test (in Bayesian Optimization this space creates a probability distribution for each of the used Hyperparameters).
#Optimization Algorithm = defines the search algorithm to use to select the best input values to use in each new iteration.

In [50]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials

In [51]:
space = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.quniform('max_depth', 10, 1200, 10),
        'max_features': hp.choice('max_features', ['auto', 'sqrt','log2', None]),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.choice('n_estimators', [10, 50, 300, 750, 1200,1300,1500])
    }


In [52]:
space

{'criterion': <hyperopt.pyll.base.Apply at 0x1ce313f4a00>,
 'max_depth': <hyperopt.pyll.base.Apply at 0x1ce313f4d60>,
 'max_features': <hyperopt.pyll.base.Apply at 0x1ce313f4760>,
 'min_samples_leaf': <hyperopt.pyll.base.Apply at 0x1ce313f44c0>,
 'min_samples_split': <hyperopt.pyll.base.Apply at 0x1ce313f4340>,
 'n_estimators': <hyperopt.pyll.base.Apply at 0x1ce313f4400>}

In [53]:
def objective(space):
    model = RandomForestClassifier(criterion = space['criterion'], max_depth = space['max_depth'],
                                 max_features = space['max_features'],
                                 min_samples_leaf = space['min_samples_leaf'],
                                 min_samples_split = space['min_samples_split'],
                                 n_estimators = space['n_estimators'], 
                                 )
    
    accuracy = cross_val_score(model, X_train, y_train, cv = 5).mean()

    # We aim to maximize accuracy, therefore we return it as a negative value
    return {'loss': -accuracy, 'status': STATUS_OK }

In [54]:
from sklearn.model_selection import cross_val_score
trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 80,
            trials= trials)


100%|███████████████████████████████████████████████| 80/80 [12:10<00:00,  9.13s/trial, best loss: -0.7230974276955885]


In [55]:
best

{'criterion': 1,
 'max_depth': 70.0,
 'max_features': 3,
 'min_samples_leaf': 9.425405483226741e-05,
 'min_samples_split': 0.20553443308741334,
 'n_estimators': 6}

In [56]:
crit = {0: 'entropy', 1: 'gini'}
feat = {0: 'auto', 1: 'sqrt', 2: 'log2', 3: None}
est = {0: 10, 1: 50, 2: 300, 3: 750, 4: 1200,5:1300,6:1500}


print(crit[best['criterion']])
print(feat[best['max_features']])
print(est[best['n_estimators']])

gini
None
1500


In [57]:

best['min_samples_leaf']

9.425405483226741e-05

In [58]:

trainedforest = RandomForestClassifier(criterion = crit[best['criterion']], max_depth = best['max_depth'], 
                                       max_features = feat[best['max_features']], 
                                       min_samples_leaf = best['min_samples_leaf'], 
                                       min_samples_split = best['min_samples_split'], 
                                       n_estimators = est[best['n_estimators']]).fit(X_train,y_train)
predictionforest = trainedforest.predict(X_test)
print(confusion_matrix(y_test,predictionforest))
print(accuracy_score(y_test,predictionforest))
print(classification_report(y_test,predictionforest))
acc5 = accuracy_score(y_test,predictionforest)

[[89 18]
 [26 21]]
0.7142857142857143
              precision    recall  f1-score   support

           0       0.77      0.83      0.80       107
           1       0.54      0.45      0.49        47

    accuracy                           0.71       154
   macro avg       0.66      0.64      0.65       154
weighted avg       0.70      0.71      0.71       154



# Genetic Algorithms

Genetic Algorithms tries to apply natural selection mechanisms to Machine Learning contexts.

Let's immagine we create a population of N Machine Learning models with some predifined Hyperparameters. We can then calculate the accuracy of each model and decide to keep just half of the models (the ones that performs best). We can now generate some offsprings having similar Hyperparameters to the ones of the best models so that go get again a population of N models. At this point we can again caltulate the accuracy of each model and repeate the cycle for a defined number of generations. In this way, just the best models will survive at the end of the process.

In [59]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
param = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(param)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [61]:
from tpot import TPOTClassifier


tpot_classifier = TPOTClassifier(generations= 5, population_size= 24, offspring_size= 12,
                                 verbosity= 2, early_stop= 12,
                                 config_dict={'sklearn.ensemble.RandomForestClassifier': param}, 
                                 cv = 4, scoring = 'accuracy')
tpot_classifier.fit(X_train,y_train)


Generation 1 - Current best internal CV score: 0.7051714625244037

Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x000001CE255244C0>
Traceback (most recent call last):
  File "C:\Users\PALLAW-PC\anaconda3\lib\_weakrefset.py", line 38, in _remove
    def _remove(item, selfref=ref(self)):
stopit.utils.TimeoutException: 



Generation 2 - Current best internal CV score: 0.7051714625244037
Generation 3 - Current best internal CV score: 0.7051714625244037
Generation 4 - Current best internal CV score: 0.7051714625244037
Generation 5 - Current best internal CV score: 0.7067948391477803
Best pipeline: RandomForestClassifier(CombineDFs(input_matrix, input_matrix), criterion=gini, max_depth=560, max_features=auto, min_samples_leaf=6, min_samples_split=14, n_estimators=200)


TPOTClassifier(config_dict={'sklearn.ensemble.RandomForestClassifier': {'criterion': ['entropy',
                                                                                      'gini'],
                                                                        'max_depth': [10,
                                                                                      120,
                                                                                      230,
                                                                                      340,
                                                                                      450,
                                                                                      560,
                                                                                      670,
                                                                                      780,
                                                                                 

In [62]:
accuracy = tpot_classifier.score(X_test, y_test)
print(accuracy)

0.7532467532467533


# Optimize hyperparameters of the model using Optuna
The hyperparameters of the above algorithm are n_estimators and max_depth for which we can try different values to see if the model accuracy can be improved. The objective function is modified to accept a trial object. This trial has several methods for sampling hyperparameters. We create a study to run the hyperparameter optimization and finally read the best hyperparameters.

In [64]:

import optuna
import sklearn.svm
def objective(trial):

    classifier = trial.suggest_categorical('classifier', ['RandomForest', 'SVC'])
    
    if classifier == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 200, 2000,10)
        max_depth = int(trial.suggest_float('max_depth', 10, 100, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth)
    else:
        c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
        
        clf = sklearn.svm.SVC(C=c, gamma='auto')

    return sklearn.model_selection.cross_val_score(
        clf,X_train,y_train, n_jobs=-1, cv=3).mean()

In [65]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2020-09-06 05:18:50,794] Trial 0 finished with value: 0.6384425314841384 and parameters: {'classifier': 'SVC', 'svc_c': 1051.6900132898652}. Best is trial 0 with value: 0.6384425314841384.
[I 2020-09-06 05:18:53,428] Trial 1 finished with value: 0.6384425314841384 and parameters: {'classifier': 'SVC', 'svc_c': 74604033.66884477}. Best is trial 0 with value: 0.6384425314841384.
[I 2020-09-06 05:18:56,992] Trial 2 finished with value: 0.6824007651841223 and parameters: {'classifier': 'RandomForest', 'n_estimators': 740, 'max_depth': 22.13807912611403}. Best is trial 2 with value: 0.6824007651841223.
[I 2020-09-06 05:19:01,122] Trial 3 finished with value: 0.6905228758169933 and parameters: {'classifier': 'RandomForest', 'n_estimators': 1120, 'max_depth': 97.781621451631}. Best is trial 3 with value: 0.6905228758169933.
[I 2020-09-06 05:19:03,162] Trial 4 finished with value: 0.6937988203411445 and parameters: {'classifier': 'RandomForest', 'n_estimators': 580, 'max_depth': 43.16565665

[I 2020-09-06 05:22:25,120] Trial 73 finished with value: 0.6970508528614697 and parameters: {'classifier': 'RandomForest', 'n_estimators': 1900, 'max_depth': 10.588840089581979}. Best is trial 28 with value: 0.7035549179021201.
[I 2020-09-06 05:22:28,686] Trial 74 finished with value: 0.695424836601307 and parameters: {'classifier': 'RandomForest', 'n_estimators': 840, 'max_depth': 10.682041764285668}. Best is trial 28 with value: 0.7035549179021201.
[I 2020-09-06 05:22:36,198] Trial 75 finished with value: 0.6905228758169933 and parameters: {'classifier': 'RandomForest', 'n_estimators': 1850, 'max_depth': 30.114145092615583}. Best is trial 28 with value: 0.7035549179021201.
[I 2020-09-06 05:22:43,076] Trial 76 finished with value: 0.6872867846325521 and parameters: {'classifier': 'RandomForest', 'n_estimators': 1970, 'max_depth': 23.714510846278433}. Best is trial 28 with value: 0.7035549179021201.
[I 2020-09-06 05:22:48,976] Trial 77 finished with value: 0.695424836601307 and parame

Accuracy: 0.7035549179021201
Best hyperparameters: {'classifier': 'RandomForest', 'n_estimators': 960, 'max_depth': 10.0015244706179}


In [66]:

trial

FrozenTrial(number=28, value=0.7035549179021201, datetime_start=datetime.datetime(2020, 9, 6, 5, 19, 54, 517166), datetime_complete=datetime.datetime(2020, 9, 6, 5, 19, 58, 217376), params={'classifier': 'RandomForest', 'n_estimators': 960, 'max_depth': 10.0015244706179}, distributions={'classifier': CategoricalDistribution(choices=('RandomForest', 'SVC')), 'n_estimators': IntUniformDistribution(high=2000, low=200, step=10), 'max_depth': LogUniformDistribution(high=100, low=10)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=28, state=TrialState.COMPLETE)

In [67]:
study.best_params

{'classifier': 'RandomForest',
 'n_estimators': 960,
 'max_depth': 10.0015244706179}

In [68]:
rf=RandomForestClassifier(n_estimators=330,max_depth=30)
rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=30, n_estimators=330)

In [69]:
y_pred=rf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[90 17]
 [25 22]]
0.7272727272727273
              precision    recall  f1-score   support

           0       0.78      0.84      0.81       107
           1       0.56      0.47      0.51        47

    accuracy                           0.73       154
   macro avg       0.67      0.65      0.66       154
weighted avg       0.72      0.73      0.72       154

